In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

from openpyxl import Workbook

In [6]:
driver = webdriver.Chrome()
url = "https://www.naver.com/"
driver.get(url)

In [7]:
def returnText(selector):
    try:
        text = driver.find_element(By.CSS_SELECTOR, selector).text
    except:
        text = None
    return(text)

In [8]:
searchBox = driver.find_element(By.XPATH,'//*[@id="query"]')
searchBox.send_keys("식당")
time.sleep(1)
searchBox.send_keys(Keys.RETURN)
map = driver.find_element(By.XPATH,'//*[@id="lnb"]/div[1]/div/div[1]/div/div[1]/div[1]/a').click()
time.sleep(1)

In [9]:
driver.switch_to.window(driver.window_handles[-1])
time.sleep(1)
iFrame = driver.find_element(By.ID,'searchIframe')
driver.switch_to.frame(iFrame)
time.sleep(0.3)


In [10]:
restaurantGroup = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')

last_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
while True:
    driver.execute_script("arguments[0].scrollBy(0, 1000)", restaurantGroup)
    new_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
    if new_height == last_height:
        break
    last_height = new_height
    time.sleep(0.1)

In [11]:
restaurants = []
i = 1
time.sleep(1)
while True:
    driver.switch_to.default_content()
    iFrame = driver.find_element(By.ID,'searchIframe')
    driver.switch_to.frame(iFrame)
    currentPage = driver.find_element(By.CLASS_NAME,"mBN2s.qxokY").text
    try: 
        restaurant = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[%d]/div[1]/a/div/div/span[1]'%i)
        restaurant.click()
        time.sleep(1)
        driver.switch_to.default_content()
        iFrame = driver.find_element(By.ID,'entryIframe')
        driver.switch_to.frame(iFrame)
        try:
            restaurantImg = driver.find_element(By.CSS_SELECTOR,'#ibu_1')
            restaurantImgSrc = restaurantImg.get_attribute('src')
        except:
            restaurantImgSrc = 'none'
        restaurantName = returnText('#_title > div > span.GHAhO')
        restaurantTags = returnText('#_title > div > span.lnJFt')
        time.sleep(1)
        restaurantAddress = returnText('#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > a > span.LDgIH')
        restaurantCallNumber = returnText('#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.nbXkr > div > span.xlx7Q')
        page=driver.find_element(By.CSS_SELECTOR,'html')
        driver.execute_script("arguments[0].scrollBy(0, 500)", page)
        time.sleep(2)
        restaurantMenu1 = returnText('#app-root > div > div > div > div:nth-child(6) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(1) > div > div.Fi0vA > div > div > a.place_bluelink.ihmWt')
        menu1Price = returnText('#app-root > div > div > div > div:nth-child(6) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(1) > div > div.mkBm3 > em')
        restaurantMenu2 = returnText('#app-root > div > div > div > div:nth-child(6) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(2) > div > div.Fi0vA > div > div > a.place_bluelink.ihmWt')
        menu2Price = returnText('#app-root > div > div > div > div:nth-child(6) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(2) > div > div.mkBm3 > em')
        restaurantMenu3 = returnText('#app-root > div > div > div > div:nth-child(6) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(3) > div > div.Fi0vA > div > div > a.place_bluelink.ihmWt')
        menu3Price = returnText('#app-root > div > div > div > div:nth-child(6) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(3) > div > div.mkBm3 > em')
        
        restaurants.append({
            'name': restaurantName,
            'tags': restaurantTags,
            'address': restaurantAddress,
            'callNumber': restaurantCallNumber,
            'imgSrc': restaurantImgSrc,
            'menu1' : restaurantMenu1,
            'price1' : menu1Price,
            'menu2' : restaurantMenu2,
            'price2' : menu2Price,
            'menu3' : restaurantMenu3,
            'price3' : menu3Price,
            
        })
        i+=1
    except:
        nextButton = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[6]')
        nextButton.click()
        time.sleep(1)
        nextPage = driver.find_element(By.CLASS_NAME,"mBN2s.qxokY").text
        if nextPage == currentPage: break
        restaurantGroup = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')
        last_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
        while True:
            driver.execute_script("arguments[0].scrollBy(0, 1000)", restaurantGroup)
            new_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
            if new_height == last_height:
                break
            last_height = new_height
            time.sleep(0.2)
        i = 1
    print(restaurants[len(restaurants)-1])

{'name': '해물상회 성남점', 'tags': '해물,생선요리', 'address': '경기 성남시 중원구 마지로 54 1층', 'callNumber': '0507-1468-9093', 'imgSrc': 'https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230516_14%2F16842239510086lR2F_JPEG%2FIMG_1583.jpeg', 'menu1': None, 'price1': None, 'menu2': None, 'price2': None, 'menu3': None, 'price3': None}
{'name': '모란술집 캠핑포차 불멍 성남동점', 'tags': '요리주점', 'address': '경기 성남시 중원구 성남대로1140번길 10-1 지하1층 모란술집 캠핑포차 불멍 성남동점', 'callNumber': '0507-1429-1438', 'imgSrc': 'none', 'menu1': None, 'price1': None, 'menu2': None, 'price2': None, 'menu3': None, 'price3': None}
{'name': '순대만찬 성남본점', 'tags': '순대,순댓국', 'address': '경기 성남시 중원구 광명로 89 성남센트럴푸르지오시티 117호, 118호', 'callNumber': '0507-1410-0977', 'imgSrc': 'https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240424_226%2F1713921236151j4DwV_JPEG%2FDSC05907.jpg', 'menu1': None, 'price1': None, 'menu2': None, 'price2': None, 'm

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.113)
Stacktrace:
0   chromedriver                        0x00000001047ea4c8 chromedriver + 4302024
1   chromedriver                        0x00000001047e2e10 chromedriver + 4271632
2   chromedriver                        0x000000010441419c chromedriver + 278940
3   chromedriver                        0x00000001043ef474 chromedriver + 128116
4   chromedriver                        0x000000010447b394 chromedriver + 701332
5   chromedriver                        0x000000010448e4e4 chromedriver + 779492
6   chromedriver                        0x000000010444b004 chromedriver + 503812
7   chromedriver                        0x000000010444b9ec chromedriver + 506348
8   chromedriver                        0x00000001047b2510 chromedriver + 4072720
9   chromedriver                        0x00000001047b6fbc chromedriver + 4091836
10  chromedriver  

In [ ]:
wb = Workbook()
ws = wb.active
ws.title = "Restaurants"
column = ['name','tags','adress','callNumber','imgSrc','menu1','price1','menu2','price2','menu3','price3']
ws.append(column)
for i in range(len(restaurants)):
    row = [restaurants[i]['name'],restaurants[i]['tags'],restaurants[i]['address'],restaurants[i]['callNumber'],restaurants[i]['imgSrc'],restaurants[i]['menu1'],restaurants[i]['price1'],restaurants[i]['menu2'],restaurants[i]['price2'],restaurants[i]['menu3'],restaurants[i]['price3']]
    ws.append(row)
wb.save('restaurant list.xlsx')
wb.close()

NameError: name 'restaurants' is not defined